In [ ]:
# Basic imports
import sys
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

#JUPYTER notebook magics
%matplotlib inline 

In [ ]:
# Creates initial conditions for the Mehlmann et al. (2021) benchmark case, at 2, 4, 8, and 16 km resolutions.

from math import sin
sys.path.append('../../nextsimdg/run')
from make_init_base import initMaker

# Domain size [km]
L = 512
# grid spacing [km]
for res in [8, 16]:

    nfirst = int(L / res)
    nsecond = int(L / res)
    nLayers = 1

    fname = f"init_benchmark_{nfirst}x{nsecond}.nc"

    initializer = initMaker(fname, nfirst, nsecond, nLayers, res*1e3, checkZeros=False)
    # The model expects everything in metres, while the benchmark problem in Mehlman et al. (2021) is defined in km.

    # Ice everywhere and all boundaries closed
    initializer.mask[:, :] = 1.
    initializer.mask[0, :] = 0.
    initializer.mask[-1, :] = 0.
    initializer.mask[:, 0] = 0.
    initializer.mask[:, -1] = 0.

    # Uniform concentration of 100%
    initializer.cice[:, :] = 1.

    # Loop over ice thickness to construct the initial conditions. This should be a pattern of undulating ice.
    for ix in range(nfirst):
        x = ix * res
        for iy in range(nsecond):
            y = iy * res
            initializer.hice[ix, iy] = 0.3 + 0.005 * (sin(60e-3 * x) + sin(30e-3 * y))

    initializer.damage[:, :] = 1.

    # All other variables are zero or not needed

In [ ]:
%%bash

# Run the  model with the benchmark config file
time /home/nextsimdg/build/nextsim --config-file config_files/config_benchmark_32x32.cfg
time /home/nextsimdg/build/nextsim --config-file config_files/config_benchmark_64x64.cfg

In [ ]:
# Load the NetCDF output files
data_32  = xr.open_dataset("benchmark_32x32.diagnostic.nc", group="/data")
data_64 = xr.open_dataset("benchmark_64x64.diagnostic.nc", group="/data")
#print(data_64)

In [ ]:
# we pick the final time-step
shear_32 = data_32['shear'][-1]
shear_64 = data_64['shear'][-1]
cice_32 = data_32['cice'][-1][:,:,-1]
cice_64 = data_64['cice'][-1][:,:,-1]
cice_min = min(np.min(cice_64), np.min(cice_32))
cice_max = max(np.max(cice_64), np.max(cice_32))

fig, axs = plt.subplots(2, 2)
cs = axs[0][0].imshow(cice_32)
cs = axs[1][0].imshow(shear_32)
#fig.colorbar(cs)
cs = axs[0][1].imshow(cice_64)
cs = axs[1][1].imshow(shear_64)
#fig.colorbar(cs)
plt.show()

In [ ]:
data_32.close()
data_64.close()